# Linear Regression Example

Let's walk through the steps of the official documentation example. Doing this will help your ability to read from the documentation, understand it, and then apply it to your own problems (the upcoming Consulting Project).

In [1]:
import findspark
findspark.init('/home/czh/spark-2.4.6-bin-hadoop2.7/')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
# Load training data
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

Interesting! We haven't seen libsvm formats before. In fact the aren't very popular when working with datasets in Python, but the Spark Documentation makes use of them a lot because of their formatting. Let's see what the training data looks like:

In [6]:
training.show(20, False)

+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label              |features                                                                                                                                                                                                                                |
+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|-9.490009878824548 |(10,[0,1,2,3,4,5,6,7,8,9],[0.4551273600657362,0.36644694351969087,-0.38256108933468047,-0.4458430198517267,0.33109790358914726,0.8067445293443565,-0.2624341731773887,-0.44850386111659524,-0.07269284838169332,0.5658

In [7]:
training.head(1)

[Row(label=-9.490009878824548, features=SparseVector(10, {0: 0.4551, 1: 0.3664, 2: -0.3826, 3: -0.4458, 4: 0.3311, 5: 0.8067, 6: -0.2624, 7: -0.4485, 8: -0.0727, 9: 0.5658}))]

This is the format that Spark expects. Two columns with the names "label" and "features". 

The "label" column then needs to have the numerical label, either a regression numerical value, or a numerical value that matches to a classification grouping. Later on we will talk about unsupervised learning algorithms that by their nature do not use or require a label.

The feature column has inside of it a vector of all the features that belong to that row. Usually what we end up doing is combining the various feature columns we have into a single 'features' column using the data transformations we've learned about.

Let's continue working through this simple example!

In [8]:
# These are the default values for the featuresCol, labelCol, predictionCol
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

# You could also pass in additional parameters for regularization, do the reading 
# in ISLR to fully understand that, after that its just some simple parameter calls.
# Check the documentation with Shift+Tab for more info!

In [9]:
# Fit the model
lrModel = lr.fit(training)

In [10]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {}".format(str(lrModel.coefficients))) # For each feature...
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

Coefficients: [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290003,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.6956151804322931]


Intercept:0.14228558260358093


There is a summary attribute that contains even more info!

In [11]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary

Lots of info, here are a few examples:

In [12]:
trainingSummary.predictions.show()

+-------------------+--------------------+--------------------+
|              label|            features|          prediction|
+-------------------+--------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|  1.5211201432720063|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...| -0.6658770747591632|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|  0.1568703823211514|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|  0.6374146679690593|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|   2.372566473232916|
| -7.896274316726144|(10,[0,1,2,3,4,5,...| -1.9410651727650883|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|  2.2621027950886363|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|-0.00134792656609...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...| -3.0051104606414007|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|  3.5437265095387804|
| -5.082010756207233|(10,[0,1,2,3,4,5,...| -0.4889664122481736|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|  1.5073098457843013|
| 14.323146365332388|(10,[0,1,2,3,4,5,..

In [13]:
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176538|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
|  2.122807193191233|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows

RMSE: 10.16309157133015
r2: 0.027839179518600154


## Train/Test Splits

But wait! We've commited a big mistake, we never separated our data set into a training and test set. Instead we trained on ALL of the data, something we generally want to avoid doing. Read ISLR and check out the theory lecture for more info on this, but remember we won't get a fair evaluation of our model by judging how well it does again on the same data it was trained on!

Luckily Spark DataFrames have an almost too convienent method of splitting the data! Let's see it:

In [14]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [15]:
# Pass in the split between training/test as a list.
# No correct, but generally 70/30 or 60/40 splits are used. 
# Depending on how much data you have and how unbalanced it is.
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [16]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
|-16.151349351277112|(10,[0,1,2,3,4,5,...|
+----------

In [17]:
test_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|
| -16.08565904102149|(10,[0,1,2,3,4,5,...|
|-12.773226999251197|(10,[0,1,2,3,4,5,...|
| -12.41094640284016|(10,[0,1,2,3,4,5,...|
|-11.039347808253828|(10,[0,1,2,3,4,5,...|
|-10.600130341909033|(10,[0,1,2,3,4,5,...|
| -10.57769830424322|(10,[0,1,2,3,4,5,...|
| -10.29566593602992|(10,[0,1,2,3,4,5,...|
|-10.293714040655924|(10,[0,1,2,3,4,5,...|
| -8.680225911784335|(10,[0,1,2,3,4,5,...|
| -8.605713514762092|(10,[0,1,2,3,4,5,...|
| -8.508663400554862|(10,[0,1,2,3,4,5,...|
+----------

In [18]:
unlabeled_data = test_data.select('features')

In [19]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



Now we only train on the train_data

In [20]:
correct_model = lr.fit(train_data)

Now we can directly get a .summary object using the evaluate method:

In [21]:
test_results = correct_model.evaluate(test_data)

In [22]:
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))

+-------------------+
|          residuals|
+-------------------+
|-27.549115093811846|
|-24.280260096159004|
|-24.236627875747494|
|-20.570205207695427|
| -20.23521696497476|
| -20.26437449689598|
| -20.20868816218606|
|-15.528863318849123|
| -18.56635948320099|
|-15.052041150079804|
| -9.236118315383543|
| -9.625868932915406|
| -8.487469333145736|
| -10.04683330122613|
| -9.807519517481511|
| -9.698492256236678|
| -6.999410619602581|
| -9.627173038977368|
| -8.705477896764423|
| -9.432904850662975|
+-------------------+
only showing top 20 rows

RMSE: 9.55374187599573


Well that is nice, but realistically we will eventually want to test this model against unlabeled data, after all, that is the whole point of building the model in the first place. We can again do this with a convenient method call, in this case, transform(). Which was actually being called within the evaluate() method. Let's see it in action:

In [23]:
predictions = correct_model.transform(unlabeled_data)

In [24]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| 0.7436316653287743|
|(10,[0,1,2,3,4,5,...|-2.4559470864427198|
|(10,[0,1,2,3,4,5,...| 1.2868019395514185|
|(10,[0,1,2,3,4,5,...| -2.267255209223915|
|(10,[0,1,2,3,4,5,...|0.36222592690635536|
|(10,[0,1,2,3,4,5,...| 0.5970558815242587|
|(10,[0,1,2,3,4,5,...| 0.8063521319715069|
|(10,[0,1,2,3,4,5,...|-2.2747628698153948|
|(10,[0,1,2,3,4,5,...|  2.304929207748261|
|(10,[0,1,2,3,4,5,...|-1.0336178909416862|
|(10,[0,1,2,3,4,5,...| -3.537108683867655|
|(10,[0,1,2,3,4,5,...| -2.785077469924754|
|(10,[0,1,2,3,4,5,...|-2.5518784751080923|
|(10,[0,1,2,3,4,5,...|-0.5532970406829028|
|(10,[0,1,2,3,4,5,...|-0.7701787867617094|
|(10,[0,1,2,3,4,5,...|-0.5971736797932425|
|(10,[0,1,2,3,4,5,...|-3.2943034210533435|
|(10,[0,1,2,3,4,5,...| 0.9469471271930322|
|(10,[0,1,2,3,4,5,...| 0.0997643820023304|
|(10,[0,1,2,3,4,5,...| 0.9242414501081134|
+----------

Okay, so this data is a bit meaningless, so let's explore this same process with some data that actually makes a little more intuitive sense!